In [2]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from Crypto.Cipher import AES
import base64
import datetime
import aescore
import cluster
import time

np.set_printoptions(formatter={'int':hex})

def encrypt(key, data):    
    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    #crypto = aes.encrypt(data)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #data = np.frombuffer(data, dtype=np.uint64)
    crypto = aes.encrypt(data)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #print np.getbuffer(data)
    #print data
    return base64.b64encode(crypto)

def dencrypt(key, cryptdata):    
    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    #crypto = aes.encrypt(data)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #data = np.frombuffer(data, dtype=np.uint64)
    crypto = aes.decrypt(cryptdata)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #print np.getbuffer(data)
    #print data
    return base64.b64encode(crypto)

def encryptfile(key, fname='files/test.txt'):
    t1 = datetime.datetime.now()
    data = np.fromfile(fname, dtype=np.uint64)  
    t2 = datetime.datetime.now()
    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    
    #print data
    for i in range(len(data)/2):
        data[2*i:2*(i+1)] = np.frombuffer(aes.encrypt(data[2*i:(2*i)+2]), dtype=np.uint64)
    
    t3 = datetime.datetime.now()
    #print data
    
    data.tofile(fname+'.enc')
    t4 = datetime.datetime.now()
    
    t5 = t2-t1
    t6 = t3-t2
    t7 = t4-t3
    print "\nFile Read time: %s seconds" % (t5.total_seconds())
    print "\nEncryption time: %s seconds" % (t6.total_seconds())
    print "\nFile Write time: %s seconds" % (t7.total_seconds())
    #with open(fname+'.enc', 'wb') as f:
    #    f.write(base64.b64encode(data))
    return


def fpgaencryptfile(key, Cluster, fname='files/test.txt'):
    t1 = datetime.datetime.now()
    data = np.fromfile(fname, dtype=np.uint64)  
    t2 = datetime.datetime.now()

    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    N = 16
    print data
    fpgadata = np.array([], dtype=np.uint64)
    for i in range(len(data)/(2*N)):
        packetid0, databuff0 = aescore.burst_singlekey(key, data[N*i:N*(i+1)])
        #packetid1, databuff1 = aescore.burst_singlekey(key, data[N*(i+1):N*(i+2)])
        #softencdata = np.frombuffer(aes.encrypt(data[N*i:N*(i+1)]), dtype=np.uint64)
        #packetid1, databuff1 = aescore.burst_singlekey(key, data[N*(i+1):N*(i+2)])
        #print packetid, databuff
        if i%2:
            Cluster.sendpackets(databuff0, source=0x0000, sink=0x0001)
        else:
            Cluster.sendpackets(databuff0, source=0x0000, sink=0x0002)
        #Cluster.sendpackets(databuff1, source=0x0000, sink=0x0002)
        for j in range(10):
            time.sleep(1/10)
            retbuff = Cluster.getpackets([packetid0])
            if len(retbuff):
                fpgadata = np.append(fpgadata, retbuff[0]['packet'][1:])
                #data[N*i:N*(i+1)] = fpgadata
                print '##################'
                #print softencdata 
                print retbuff[0]['packet'] #retbuff[1]['packet']
                break
    print fpgadata
    
    t3 = datetime.datetime.now()
    #print data
    
    fpgadata.tofile(fname+'.enc')
    t4 = datetime.datetime.now()
    
    t5 = t2-t1
    t6 = t3-t2
    t7 = t4-t3
    print "\nFile Read time: %s seconds" % (t5.total_seconds())
    print "\nEncryption time: %s seconds" % (t6.total_seconds())
    print "\nFile Write time: %s seconds" % (t7.total_seconds())
    #with open(fname+'.enc', 'wb') as f:
    #    f.write(base64.b64encode(data))
    return

def fpgaencryptfile1(key, Cluster, fname='files/test.txt'):
    t1 = datetime.datetime.now()
    data = np.fromfile(fname, dtype=np.uint64) 
    N = 32
    M = 2
    #print data
    fpgadata = np.array([], dtype=np.uint64)
    pad =  np.append(np.array([N - len(data)%N], dtype=np.uint64),
                     np.zeros(N - len(data)%N + M*N -1, dtype=np.uint64))
    data = np.append(data, pad) 
    t2 = datetime.datetime.now()
    #print data
    
    packetid = []
    databuff = []
    for i in range(len(data)/N):
        pktid, databuff = aescore.burst_singlekey(key, data[N*i:N*(i+1)])
        packetid.append(pktid)
        if i%2:
            Cluster.sendpackets(databuff, source=0x0000, sink=0x0001)
        else:
            Cluster.sendpackets(databuff, source=0x0000, sink=0x0002)
        #print packetid
        for j in range(10):
            #time.sleep(1/50)
            retbuff = Cluster.getpackets([pktid])
            #print retbuff
            for i in retbuff:
                packetid.remove(i['packetid'])
                fpgadata = np.append(fpgadata, i['packet'][1:])
                #print '#',
                #print softencdata 
                #print retbuff[0]['packet'] #retbuff[1]['packet']
            if len(packetid) == 0:
                break
    
    t3 = datetime.datetime.now()
    #print data
    
    fpgadata.tofile(fname+'.enc')
    t4 = datetime.datetime.now()
    
    t5 = t2-t1
    t6 = t3-t2
    t7 = t4-t3
    print "\nFile Read time: %s seconds" % (t5.total_seconds())
    print "\nEncryption time: %s seconds" % (t6.total_seconds())
    print "\nFile Write time: %s seconds" % (t7.total_seconds())
    #with open(fname+'.enc', 'wb') as f:
    #    f.write(base64.b64encode(data))
    return


def decryptfile(key, fname='files/test.txt.enc'):
    t1 = datetime.datetime.now()
    data = np.fromfile(fname, dtype=np.uint64)  
    t2 = datetime.datetime.now()
    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    
    #print data
    for i in range(len(data)/2):
        data[2*i:2*(i+1)] = np.frombuffer(aes.decrypt(data[2*i:(2*i)+2]), dtype=np.uint64)
    
    t3 = datetime.datetime.now()
    #print data
    #print len(data)
    padsize = 0
    padindex = 0
    for index,value in enumerate(reversed(data)):
        if value != 0:
            #print index,value
            padsize = value
            padindex = index
            break
    data = data[:-padindex-1]
    #print data
    
    
    data.tofile(fname+'.dec')
    t4 = datetime.datetime.now()
    
    t5 = t2-t1
    t6 = t3-t2
    t7 = t4-t3
    print "\nFile Read time: %s seconds" % (t5.total_seconds())
    print "\nDecryption time: %s seconds" % (t6.total_seconds())
    print "\nFile Write time: %s seconds" % (t7.total_seconds())
    return

def comparefile(origfile, newfile):
    data0 = np.fromfile(origfile, dtype=np.uint64)
    data1 = np.fromfile(newfile, dtype=np.uint64)
    print len(data0), len(data1)
    i = 0
    for j, k in zip(data0, data1):
        if j != k:
            print i, j, k
        i += 1
    return

In [3]:
#comparefile('files/test.pdf', 'files/test.pdf.enc.dec.pdf')

In [4]:
Cluster = cluster.cluster()
Cluster.conf(1)

key = np.array([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000], dtype=np.uint64) 
fpgaencryptfile1(key, Cluster, fname='files/flower.png')

PCI is being Initialized....
PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000
pci_status : 0xffffffff001001feL
pci_command : 0xffffffff000000feL
pci_status : 0x100100L
pci_command : 0x100100L
setting node 0 with address : 0

File Read time: 0.00113 seconds

Encryption time: 56.317311 seconds

File Write time: 0.000707 seconds


In [6]:
decryptfile(key, fname='files/flower.png.enc')


File Read time: 0.000405 seconds

Decryption time: 0.157074 seconds

File Write time: 0.000924 seconds


In [ ]:
key = np.array([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000], dtype=np.uint64) 
encryptfile(key, fname='files/big_buck_bunny_480p_h264.mov')

In [37]:
decryptfile(key, fname='files/big_buck_bunny_480p_h264.mov.enc')


File Read time: 0.08067 seconds

Decryption time: 48.573275 seconds

File Write time: 0.151388 seconds


In [53]:
a = np.array([0x00000000002000000], dtype=np.uint64) 
b = np.array([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000], dtype=np.uint64) 
c = np.array([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000], dtype=np.uint64) 
print np.append(a, [b, c])

[0x2000000L 0x0L 0x0L 0x0L 0x0L 0x0L 0x0L 0x0L 0x0L]
